In [0]:
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

Before Convolution 1 = [50 28 28  1]
After Convolution 1 =  [50 28 28 32]
After Max Pooling 1 =  [50 14 14 32]
After Convolution 2 =  [50 14 14 64]
After Max Pooling 2 =  [50  7  7 64]
After flattening =     [  50 3136]
fully connected layer of 3136 connected to 1024 nodes =  [  50 1024]


In [0]:
init =tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  print(sess.run(x_image))

In [0]:
mnist = input_data.read_data_sets("MNIST_DATA/", one_hot =True)

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [0]:
def init_weights(shape):
  init_random_dist = tf.truncated_normal(shape, stddev= 0.1)
  return tf.Variable(init_random_dist)

In [0]:
def init_bias(shape):
  init_bias_vals = tf.constant(0.1,shape=shape)
  return tf.Variable(init_bias_vals)

In [0]:
def conv2d(x,W):
  return tf.nn.conv2d(x,W, strides = [1,1,1,1],padding= 'SAME')

In [0]:
def max_pool_2by2(x):
  return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding ='SAME')


In [0]:
def convolutional_layer(input_x,shape):
  W = init_weights(shape)
  b = init_bias([shape[3]])
  return tf.nn.relu(conv2d(input_x,W)+b)

In [0]:
def normal_full_layer(input_layer,size):
  input_size=int(input_layer.get_shape()[1])
  W =init_weights([input_size, size])
  b =init_bias([size])
  return tf.matmul(input_layer,W)+b

In [0]:
x =tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [0]:
x_image =tf.reshape(x,[-1,28,28,1])

Before Convolution 1 = [50 28 28  1]
After Convolution 1 =  [50 28 28 32]
After Max Pooling 1 =  [50 14 14 32]
After Convolution 2 =  [50 14 14 64]
After Max Pooling 2 =  [50  7  7 64]
After flattening =     [  50 3136]
fully connected layer of 3136 connected to 1024 nodes =  [  50 1024]


[[4. 3. 1. 0.]
 [2. 1. 0. 1.]
 [1. 2. 4. 1.]
 [3. 1. 0. 2.]]


#Layer

In [0]:
convo_1 =convolutional_layer(x_image,shape = [5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [0]:
sha = [5,5,1,32]
sha[3]

32

In [0]:
convo_2 =convolutional_layer(convo_1_pooling,shape = [5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [0]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [0]:
hold_prob =tf.placeholder(tf.float32)
full_one_dropout =tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

W0722 17:47:07.054459 140421296306048 deprecation.py:506] From <ipython-input-15-7e8cb18c2b5d>:2: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
y_pred =normal_full_layer(full_one_dropout,10)

Loss function

In [0]:
cross_entropy =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels =y_true,logits=y_pred))

Optimizer

In [0]:
optimizer =tf.train.AdamOptimizer(learning_rate=0.0001)
train =optimizer.minimize(cross_entropy)

Initalize Variables

In [0]:
init =tf.global_variables_initializer()

Session

In [0]:
steps =2000

with tf.Session() as sess:
  sess.run(init)
  
  for i in range(steps):
    batch_x, batch_y = mnist.train.next_batch(50)
    
    sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
    
    #print out a message every 100 steps
    if i%100 ==0:
      print('Currently on step {}'.format(i))
      print('Accuracy is:')
      #Test the Train Model
      matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
      
      acc= tf.reduce_mean(tf.cast(matches,tf.float32))
      
      print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
      print('\n')
      
      

In [0]:
#Explaination
batch_x, batch_y = mnist.train.next_batch(50)

x_image =tf.reshape(batch_x,[-1,28,28,1])
# Build a covo1.
W1= tf.truncated_normal([5,5,1,32], stddev= 0.1)
b1 = init_bias([32])
con1= tf.nn.conv2d(x_image, W1, [1, 1, 1, 1], "SAME")
con1=tf.nn.relu(con1+b1)

# Build a maxpool layer 1.
con1_pooling = max_pool_2by2(con1)

# Build a covo2.
W2= tf.truncated_normal([5,5,32,64], stddev= 0.1)
b2 = init_bias([64])
con2= tf.nn.conv2d(con1_pooling, W2, [1, 1, 1, 1], "SAME")
con2=tf.nn.relu(con2+b2)

# Build a maxpool layer 1.
con2_pooling = max_pool_2by2(con2)

con2_flat = tf.reshape(con2_pooling,[-1,7*7*64])

full_layer = tf.nn.relu(normal_full_layer(con2_flat,1024))

e=int(con2_flat.get_shape()[1])

# Launch the graph in a session.
with tf.Session() as sess:
  print("Before Convolution 1 =",sess.run(tf.shape(x_image)))
  print("After Convolution 1 = ",sess.run(tf.shape(con1)))
  print("After Max Pooling 1 = ",sess.run(tf.shape(con1_pooling)))
  print("After Convolution 2 = ",sess.run(tf.shape(con2)))
  print("After Max Pooling 2 = ",sess.run(tf.shape(con2_pooling)))
  print("After flattening =    ",sess.run(tf.shape(con2_flat)))
  print("fully connected layer of 3136 connected to 1024 nodes = ",sess.run(tf.shape(full_layer)))

Before Convolution 1 = [50 28 28  1]
After Convolution 1 =  [50 28 28 32]
After Max Pooling 1 =  [50 14 14 32]
After Convolution 2 =  [50 14 14 64]
After Max Pooling 2 =  [50  7  7 64]
After flattening =     [  50 3136]
fully connected layer of 3136 connected to 1024 nodes =  [  50 1024]


In [0]:
#Eg of cnn
import tensorflow as tf
k = tf.constant([
    [1, 0, 1],
    [2, 1, 0],
    [0, 0, 1]
], dtype=tf.float32, name='k')
i = tf.constant([
    [4, 3, 1, 0],
    [2, 1, 0, 1],
    [1, 2, 4, 1],
    [3, 1, 0, 2]
], dtype=tf.float32, name='i')
kernel = tf.reshape(k, [3, 3, 1, 1], name='kernel')
image  = tf.reshape(i, [1, 4, 4, 1], name='image')


res = tf.squeeze(tf.nn.conv2d(image, kernel, [1, 1, 1, 1], "VALID"))
# VALID means no padding
with tf.Session() as sess:
   print(sess.run(tf.squeeze(res)))

[[14.  6.]
 [ 6. 12.]]
